In [ ]:
import numpy as np
import param
import panel as pn

pn.extension()

In [ ]:
def link_param_method(obj: param.Parameterized, target: str, source: str):
    """Link a parameterized method to a target parameter.

    When the dependencies of the source method change, set the target
    parameter to the return value of the source method.
    """

    def update(*events):
        setattr(obj, target, getattr(obj, source)())

    for dep in obj.param.params_depended_on(source):
        (dep.inst or dep.cls).param.watch(update, dep.name, dep.what)

    update()

In [ ]:
class Sine(param.Parameterized):
    phase = param.Number(default=0, bounds=(0, np.pi))
    frequency = param.Number(default=1, bounds=(0.1, 2))
    
    def __init__(self, **params):
        super().__init__(**params)
        link_param_method(self, "data", "compute_data")
    
    @param.depends("phase", "frequency")
    def compute_data(self):
        return np.sin(np.linspace(0, np.pi * 3, 40) * self.frequency + self.phase)
    
    data = param.Parameter(precedence=-1)

    @param.depends("data")
    def view(self):
        y = self.data
        y = ((y - y.min()) / y.ptp()) * 20
        array = np.array(
            [list((' ' * (int(round(d)) - 1) + '*').ljust(20)) for d in y])
        return pn.pane.Str('\n'.join([''.join(r) for r in array.T]), height=380, width=500)
    
    @param.depends("data")
    def summary(self):
        return pn.pane.Str(
            f"Argmax: {np.argmax(self.data)}\n"
            f"Argmin: {np.argmin(self.data)}\n"
            f"Mean: {np.mean(self.data):.3f}\n"
            f"Variance: {np.var(self.data):.3f}\n"
        )


sine = Sine(name='ASCII Sine Wave')
pn.Row(sine.param, sine.view, sine.summary)